In [1]:
# ==========================
# 🧠 CLASIFICACIÓN TERAPÉUTICA HÍBRIDA
# ==========================

import pandas as pd
import re
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from collections import Counter

# ==========================
# 1️⃣ CARGAR DATASET
# ==========================
df = pd.read_csv("../output/dataset_farmacia_limpio.csv")

c:\Users\Jeimi\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ==========================
# 2️⃣ LIMPIAR NOMBRES
# ==========================
def limpiar_nombre(nombre):
    nombre = str(nombre).upper()
    nombre = re.sub(r'[^A-ZÁÉÍÓÚÑ\s]', ' ', nombre)  # quitar símbolos y números
    nombre = re.sub(r'\s+', ' ', nombre).strip()
    return nombre

df["nombre_limpio"] = df["nombre"].apply(limpiar_nombre)

In [3]:
# ==========================
# 🩺 DICCIONARIO TERAPÉUTICO GENERALIZADO
# ==========================

diccionario_terapeutico = {
    # ------------------------------
    # 💊 Analgésicos / Antipiréticos
    # ------------------------------
    "PARACETAMOL": "Analgésico / Antipirético",
    "IBUPROFENO": "Analgésico / Antiinflamatorio",
    "DICLOFENACO": "Analgésico / Antiinflamatorio",
    "KETOROL": "Analgésico / Antiinflamatorio",
    "NOVADOL": "Analgésico / Antiinflamatorio",
    "FLAMAX": "Analgésico / Antiinflamatorio",
    "FLOGENE": "Analgésico / Antiinflamatorio",
    "DOLO OCTIRONA": "Analgésico / Antiinflamatorio",
    "MIGRANOL": "Analgésico / Antiinflamatorio",
    "CAFIAASPIRINA": "Analgésico / Antipirético",

    # ------------------------------
    # 🧫 Antibióticos
    # ------------------------------
    "AMOXICILINA": "Antibiótico",
    "AMOXIDAL": "Antibiótico",
    "AZITROMICINA": "Antibiótico",
    "CLARITROMICINA": "Antibiótico",
    "CEFTRIAXONA": "Antibiótico",
    "CIPROFLOXACINO": "Antibiótico",
    "AMPICILINA": "Antibiótico",
    "MAXIBIOTIC": "Antibiótico",

    # ------------------------------
    # 💉 Corticoides / Antiinflamatorios
    # ------------------------------
    "DEXAMETASONA": "Corticoide / Antiinflamatorio",
    "PREDNISONA": "Corticoide / Antiinflamatorio",
    "BETAMETASONA": "Corticoide / Antiinflamatorio",

    # ------------------------------
    # 🤧 Antigripales / Resfrío
    # ------------------------------
    "TAPSIN": "Antigripal / Resfrío",
    "TABCIN": "Antigripal / Resfrío",
    "FLAVICOLD": "Antigripal / Resfrío",
    "TYPIREC": "Antigripal / Resfrío",
    "REFRIANEX": "Antigripal / Resfrío",
    "DOLO OCTIRONA": "Antigripal / Analgésico combinado",

    # ------------------------------
    # 🍽️ Gastrointestinales
    # ------------------------------
    "OMEPRAZOL": "Antiácido / Protector gástrico",
    "RANITIDINA": "Antiácido / Protector gástrico",
    "LOPERAMIDA": "Antidiarreico",
    "DOMPERIDONA": "Antiemético / Regulador digestivo",
    "ESOFAX": "Antiácido / Gastrointestinal",
    "ESPASMO DIOXADO": "Antiespasmódico / Gastrointestinal",
    "FIBRASOL": "Fibra / Regulador intestinal",

    # ------------------------------
    # 💪 Vitaminas / Suplementos
    # ------------------------------
    "VITAMINA C": "Vitamina / Suplemento nutricional",
    "COMPLEJO B": "Vitamina / Suplemento nutricional",
    "NUCLEO CMP": "Vitamina / Neurometabólico",
    "MAGNESIO": "Mineral / Suplemento",
    "CALCIO": "Mineral / Suplemento",
    "COBA VIMIN": "Vitamina / Suplemento",
    "BIOELECTRO": "Rehidratante / Suplemento",
    "PROPOLEO": "Naturista / Inmunoestimulante",
    "NERVINETAS": "Vitamina / Neurometabólico",

    # ------------------------------
    # 🧴 Dermatológicos / Bálsamos / Tópicos
    # ------------------------------
    "MENTISAN": "Bálsamo / Descongestionante tópico",
    "VARNGEL": "Analgésico tópico / Antiinflamatorio",
    "VASELINA": "Emoliente / Protector dérmico",
    "CREMA": "Dermatológico general",
    "POMADA": "Dermatológico / Antiinflamatorio tópico",
    "BLOQUEADOR SOLAR": "Fotoprotector / Dermatológico",
    "DERMOVATE": "Corticoide tópico / Dermatológico",
    "ALOE VERA": "Natural / Regenerador dérmico",

    # ------------------------------
    # 🧼 Higiene / No farmacéutico
    # ------------------------------
    "KLEENEX": "Higiene / Papel sanitario",
    "ALGODON": "Higiene / Aseo personal",
    "ALCOHOL": "Antiséptico / Desinfectante",
    "CEPILLO DENTAL": "Higiene / Dental",
    "PASTA DENTAL": "Higiene / Dental",
    "ENJUAGUE BUCAL": "Higiene / Dental",
    "GUANTES": "Higiene / Protección médica",
    "JERINGA": "Material médico / Desechable",
    "COTONETE": "Higiene / Limpieza personal",
    "TALCO": "Higiene / Corporal",
    "MASCULAN": "Preservativo / Salud sexual",
    "TOALLAS HIGIÉNICAS": "Higiene femenina / Absorbentes",
    "PRESERVATIVO": "Salud sexual / Anticonceptivo",
    "GORRO": "Higiene / Protección médica",
    "BARBIJO": "Higiene / Protección respiratoria",

    # ------------------------------
    # 🌿 Naturistas / Homeopáticos
    # ------------------------------
    "STEVIA": "Natural / Endulzante",
    "MIEL": "Natural / Fortificante",
    "JENGIBRE": "Natural / Digestivo / Inmunitario",
    "MENTA": "Natural / Digestivo / Calmante",
    "PROPOLEO": "Natural / Inmunoestimulante",

    # ------------------------------
    # ❓ Otros / No clasificado
    # ------------------------------
    "RESOTYL": "Ansiolítico / Sedante",
    "TAVOR": "Ansiolítico",
    "COFARETIC": "Descongestivo urinario",
    "SEPTICIDE": "Antiséptico / Desinfectante",
    "CERTEC": "Antialérgico / Antihistamínico",
}


In [4]:
# ==========================
# 4️⃣ CLASIFICACIÓN DIRECTA CON DICCIONARIO
# ==========================
def clasificar_diccionario(nombre):
    for palabra, categoria in diccionario_terapeutico.items():
        if palabra in nombre:
            return categoria
    return None

df["categoria_diccionario"] = df["nombre_limpio"].apply(clasificar_diccionario)


In [5]:
# ==========================
# 5️⃣ EMBEDDINGS SEMÁNTICOS (usa modelo multilingüe)
# ==========================
modelo = SentenceTransformer("distiluse-base-multilingual-cased-v1")
embeddings = modelo.encode(df["nombre_limpio"].tolist(), show_progress_bar=True)


Batches: 100%|██████████| 4081/4081 [37:53<00:00,  1.80it/s] 


In [ ]:
# ==========================
# 6️⃣ CLUSTERING SEMÁNTICO
# ==========================
# Determina número de clusters (puedes ajustar n_clusters según la diversidad)
kmeans = KMeans(n_clusters=12, random_state=42)        
df["cluster"] = kmeans.fit_predict(embeddings)


In [7]:
# ==========================
# 7️⃣ ETIQUETAR CLUSTERS USANDO EL DICCIONARIO
# ==========================
def etiqueta_cluster(cluster_id):
    subset = df[df["cluster"] == cluster_id]["nombre_limpio"].tolist()
    for palabra, categoria in diccionario_terapeutico.items():
        if any(palabra in nombre for nombre in subset):
            return categoria
    return "Otros / Sin categoría"

df["categoria_cluster"] = df["cluster"].apply(etiqueta_cluster)

In [8]:
# ==========================
# 8️⃣ COMBINAR RESULTADOS
# ==========================
df["categoria_final"] = df["categoria_diccionario"].combine_first(df["categoria_cluster"])


In [9]:
# ==========================
# 9️⃣ REASIGNAR "OTROS" SEGÚN SIMILITUD SEMÁNTICA (versión segura)
# ==========================
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Extraer categorías únicas del diccionario
categorias_diccionario = list(set(diccionario_terapeutico.values()))

# Crear embeddings para cada categoría del diccionario
embeddings_categorias = modelo.encode(categorias_diccionario)

# Identificar filas sin categoría (Otros / Sin categoría)
mask_otros = df["categoria_final"].isin([None, "Otros / Sin categoría"])
otros_indices = df[mask_otros].index

print(f"🔍 Reasignando {len(otros_indices)} productos sin categoría...")

if len(otros_indices) == 0:
    print("✅ No hay productos sin categoría, se omite esta parte.")
else:
    # Embeddings de esos productos
    textos_otros = df.loc[otros_indices, "nombre_limpio"].tolist()
    embeddings_otros = modelo.encode(textos_otros)

    # Asegurar formato 2D
    if len(np.array(embeddings_otros).shape) == 1:
        embeddings_otros = np.array(embeddings_otros).reshape(1, -1)
    if len(np.array(embeddings_categorias).shape) == 1:
        embeddings_categorias = np.array(embeddings_categorias).reshape(1, -1)

    # Verificar que haya embeddings válidos
    if len(embeddings_otros) == 0 or len(embeddings_categorias) == 0:
        print("⚠️ No hay embeddings válidos para calcular similitud.")
    else:
        # Calcular similitud coseno
        similitudes = cosine_similarity(embeddings_otros, embeddings_categorias)

        # Asignar la categoría más similar
        categorias_asignadas = [
            categorias_diccionario[i] for i in np.argmax(similitudes, axis=1)
        ]

        # Actualizar el DataFrame
        df.loc[otros_indices, "categoria_final"] = categorias_asignadas
        print(f"✅ Se asignaron {len(categorias_asignadas)} categorías automáticamente.")


🔍 Reasignando 0 productos sin categoría...
✅ No hay productos sin categoría, se omite esta parte.


In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# ==========================
# 🔄 UNIFICACIÓN SEMÁNTICA DE CATEGORÍAS SIMILARES
# ==========================
print("\n🔍 Analizando similitud semántica entre categorías...")

# Obtener lista única de categorías finales
categorias_unicas = sorted(df["categoria_final"].unique())

# Calcular embeddings de cada categoría
embeddings_categorias_finales = modelo.encode(categorias_unicas)

# Calcular matriz de similitud coseno
sim_matrix = cosine_similarity(embeddings_categorias_finales)

# Definir umbral de similitud (puedes ajustar entre 0.80 y 0.95)
umbral = 0.88

# Agrupar categorías similares
grupos = []
usadas = set()

for i, cat in enumerate(categorias_unicas):
    if cat in usadas:
        continue
    similares = [
        categorias_unicas[j]
        for j in range(len(categorias_unicas))
        if sim_matrix[i, j] >= umbral
    ]
    grupos.append(similares)
    usadas.update(similares)

# Mostrar grupos detectados
print("\n🧩 Grupos de categorías similares detectados:")
for g in grupos:
    if len(g) > 1:
        print(" → ", g)

# Crear un mapeo para unificación (usa la primera del grupo como representante)
mapeo_unificacion = {cat: grupo[0] for grupo in grupos for cat in grupo}

# Aplicar al DataFrame
df["categoria_final_unificada"] = df["categoria_final"].map(mapeo_unificacion)

# ==========================
# 💾 GUARDAR RESULTADO UNIFICADO
# ==========================
df.to_csv("../output/dataset_farmacia_categorizado_unificado.csv", index=False)

print("\n✅ Archivo guardado como 'dataset_farmacia_categorizado_unificado.csv'")
print("\n📊 Distribución de categorías unificadas:")
print(df["categoria_final_unificada"].value_counts())
print(f"\nNúmero total de categorías (tras unificación): {df['categoria_final_unificada'].nunique()}")



🔍 Analizando similitud semántica entre categorías...

🧩 Grupos de categorías similares detectados:
 →  ['Analgésico / Antiinflamatorio', 'Analgésico / Antipirético', 'Analgésico tópico / Antiinflamatorio']
 →  ['Antiespasmódico / Gastrointestinal', 'Antiácido / Gastrointestinal']
 →  ['Higiene / Aseo personal', 'Higiene / Limpieza personal']
 →  ['Natural / Endulzante', 'Natural / Fortificante', 'Natural / Inmunoestimulante']
 →  ['Vitamina / Suplemento', 'Vitamina / Suplemento nutricional']

✅ Archivo guardado como 'dataset_farmacia_categorizado_unificado.csv'

📊 Distribución de categorías unificadas:
categoria_final_unificada
Analgésico / Antiinflamatorio              79441
Antigripal / Resfrío                       17254
Antibiótico                                 7319
Corticoide / Antiinflamatorio               4551
Vitamina / Suplemento                       3773
Natural / Endulzante                        2558
Antiácido / Protector gástrico              2399
Bálsamo / Descongest

In [11]:
# ==========================
# 🔟 GUARDAR RESULTADO FINAL
# ==========================
df.to_csv("../output/dataset_farmacia_categorizado.csv", index=False)
print("✅ Clasificación completada y limpiada. Archivo guardado como 'dataset_farmacia_categorizado.csv'.")

# Mostrar resumen final
print("\n📊 Categorías finales:")
print(df["categoria_final"].value_counts())
print(f"\nForma final: {df.shape[0]} filas x {df.shape[1]} columnas")

✅ Clasificación completada y limpiada. Archivo guardado como 'dataset_farmacia_categorizado.csv'.

📊 Categorías finales:
categoria_final
Analgésico / Antipirético                  59577
Analgésico / Antiinflamatorio              19837
Antigripal / Resfrío                       17254
Antibiótico                                 7319
Corticoide / Antiinflamatorio               4551
Vitamina / Suplemento nutricional           3234
Antiácido / Protector gástrico              2399
Bálsamo / Descongestionante tópico          1662
Natural / Inmunoestimulante                 1464
Material médico / Desechable                1360
Dermatológico general                       1179
Higiene / Protección respiratoria           1042
Natural / Fortificante                      1035
Higiene / Protección médica                  960
Salud sexual / Anticonceptivo                903
Antiséptico / Desinfectante                  879
Preservativo / Salud sexual                  828
Higiene / Dental              

In [12]:
print(f"\nNúmero total de categorías (tras unificación): {df['categoria_final_unificada'].nunique()}")



Número total de categorías (tras unificación): 35


In [13]:
# ==========================
# 1️⃣1️⃣ AJUSTAR A SOLO LAS CATEGORÍAS DEL DICCIONARIO ORIGINAL
# ==========================
print("\n🧠 Ajustando categorías finales solo a las del diccionario...")

# Lista de categorías válidas (las del diccionario)
categorias_diccionario_validas = list(set(diccionario_terapeutico.values()))

# Detectar las categorías que no están en el diccionario
categorias_actuales = df["categoria_final_unificada"].unique().tolist()
categorias_extra = [c for c in categorias_actuales if c not in categorias_diccionario_validas]

print(f"🔍 Se detectaron {len(categorias_extra)} categorías fuera del diccionario:")
print(categorias_extra)

if categorias_extra:
    # Crear embeddings de categorías válidas y extra
    embeddings_validas = modelo.encode(categorias_diccionario_validas)
    embeddings_extra = modelo.encode(categorias_extra)

    # Calcular similitud coseno
    similitudes = cosine_similarity(embeddings_extra, embeddings_validas)

    # Encontrar la categoría válida más parecida para cada categoría extra
    indices_mas_similares = similitudes.argmax(axis=1)
    mapping_reasignacion = {
        cat_extra: categorias_diccionario_validas[idx]
        for cat_extra, idx in zip(categorias_extra, indices_mas_similares)
    }

    # Reemplazar en el DataFrame
    df["categoria_final_ajustada"] = df["categoria_final_unificada"].replace(mapping_reasignacion)
else:
    # Si no hay categorías extra, copiamos directamente
    df["categoria_final_ajustada"] = df["categoria_final_unificada"]

# ==========================
# 💾 GUARDAR RESULTADO FINAL
# ==========================
df.to_csv("../output/dataset_farmacia_categorizado_final.csv", index=False)
print("\n✅ Archivo final guardado como 'dataset_farmacia_categorizado_final.csv'")

print("\n📊 Distribución final de categorías ajustadas:")
print(df["categoria_final_ajustada"].value_counts())
print(f"\nNúmero total de categorías (final): {df['categoria_final_ajustada'].nunique()}")



🧠 Ajustando categorías finales solo a las del diccionario...
🔍 Se detectaron 0 categorías fuera del diccionario:
[]

✅ Archivo final guardado como 'dataset_farmacia_categorizado_final.csv'

📊 Distribución final de categorías ajustadas:
categoria_final_ajustada
Analgésico / Antiinflamatorio              79441
Antigripal / Resfrío                       17254
Antibiótico                                 7319
Corticoide / Antiinflamatorio               4551
Vitamina / Suplemento                       3773
Natural / Endulzante                        2558
Antiácido / Protector gástrico              2399
Bálsamo / Descongestionante tópico          1662
Material médico / Desechable                1360
Dermatológico general                       1179
Higiene / Protección respiratoria           1042
Higiene / Protección médica                  960
Salud sexual / Anticonceptivo                903
Antiséptico / Desinfectante                  879
Preservativo / Salud sexual                  828
Hig